In [1]:
#EXPLORE WLASL
import sys
sys.path.append('.')

import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from pathlib import Path
from WLASL.start_kit import preprocess
from tqdm import tqdm

JSON_FILE_PATH = "WLASL/start_kit/WLASL_v0.3.json"
RAW_VIDEOS_PATH = Path("WLASL/start_kit/raw_videos")
SUBSET = 10

In [2]:
with open(JSON_FILE_PATH, 'r') as metadata_file:
    annotations = json.load(metadata_file)

In [60]:
def get_video_path(video_id):
    video_id = str(video_id)
    for file in RAW_VIDEOS_PATH.iterdir():
        if file.is_file() and file.stem == video_id:
            return file
    return None

In [63]:

min = 90
max = 0
for word in tqdm(annotations, desc="Getting frame counts: "):
    for instance in word["instances"]:
        video_path = get_video_path(instance["video_id"])
        if video_path:
            try:
                FPS = VideoFileClip(str(video_path)).reader.nframes
                min = min(min, FPS)
                max = max(max, FPS)
            except:
                pass
print(min, max)

Getting frame counts:   1%|          | 11/2000 [01:11<3:35:06,  6.49s/it]


KeyboardInterrupt: 

In [ ]:
#Draws bbox on video file - need instance from WLASL json file and a destination path
def bbox_annotate(instance, destination_path=None):
    if not destination_path:
        default_dir = Path("test_videos")
        if not default_dir.is_dir():
            default_dir.mkdir()

        destination_path = Path(f"test_videos/{instance['video_id']}_boxed.mp4")
    
    raw_video_path = get_video_path(instance['video_id'])
    frames = preprocess.extract_frame_as_video(str(raw_video_path), instance["frame_start"], instance["frame_end"]-1)

    x, y, w, h = instance["bbox"]
    
    for frame in frames:
        cv2.rectangle(frame, (x, y), (w, h), color=(0, 255, 0), thickness=5)
    
    width, height = VideoFileClip(str(raw_video_path)).size
    print(width, height)
    bbox_video = preprocess.convert_frames_to_video(frames, destination_path, size=(width, height))


bbox_annotate(annotations[0]["instances"][10])

WLASL/start_kit/raw_videos/07069.mp4
1280 720


In [ ]:
def bbox_crop(instance, destination_path=None):
    if not destination_path:
        default_dir = Path("test_videos")
        if not default_dir.is_dir():
            default_dir.mkdir()

        destination_path = Path(f"test_videos/{instance['video_id']}.mp4")
        
    raw_video_path = get_video_path(instance['video_id'])
    frames = preprocess.extract_frame_as_video(str(raw_video_path), instance["frame_start"], instance["frame_end"]-1)

    x, y, w, h = instance["bbox"]
    width, height = VideoFileClip(str(raw_video_path)).size

    padding_frame = np.zeros((height, width, 3), dtype=np.uint8)
    print(padding_frame.shape)
    
    cropped_frames = []
    for frame in frames:
        cropped_frame = frame[y:h, x:w]
        frame = np.copy(padding_frame)
        
        #Calculate Centering Coordinates
        pad_x = max(0, (width - abs(x - w)) // 2)
        pad_y = max(0, (height - abs(y - h)) // 2)

        #Apply Crop
        frame[pad_y:pad_y + abs(y - h), pad_x:pad_x + abs(x - w)] = cropped_frame

        cropped_frames.append(frame)
    
    bbox_video = preprocess.convert_frames_to_video(cropped_frames, destination_path, size=(width, height))
    

# bbox_annotate(annotations[69]["instances"][5])
bbox_crop(annotations[69]["instances"][5])

WLASL/start_kit/raw_videos/43171.mp4
(480, 640, 3)


In [9]:
RAW_VIDEOS_PATH = Path("WLASL/start_kit/raw_videos_mp4")
YT_VID_PATH = RAW_VIDEOS_PATH / "Kwvw-K6GYW8.mp4"

YT_VID_PATH = "test_videos/output.mp4"

def bbox_annotate_yt(path, instance, destination_path=None):

    if not destination_path:
        default_dir = Path("test_videos")
        if not default_dir.is_dir():
            default_dir.mkdir()

        destination_path = Path(f"test_videos/{instance['video_id']}.mp4")

    frames = preprocess.extract_frame_as_video(str(path), instance["frame_start"], instance["frame_end"]-1)

    x, y, w, h = instance["bbox"]

    for frame in frames:
        cv2.rectangle(frame, (x, y), (w, h), color=(0, 255, 0), thickness=5)

    width, height = VideoFileClip(str(path)).size
    print(width, height)
    bbox_video = preprocess.convert_frames_to_video(frames, destination_path, size=(width, height))

bbox_annotate_yt(YT_VID_PATH, annotations[0]["instances"][31])

#ffmpeg -i WLASL/start_kit/raw_videos_mp4/Kwvw-K6GYW8.mp4 -vf scale=1280:720 test_videos/output.mp4

#ffmpeg -i WLASL/start_kit/raw_videos_mp4/Kwvw-K6GYW8.mp4 -vf scale=1280:720 WLASL/start_kit/raw_videos_mp4/Kwvw-K6GYW8.mp4
#ffmpeg -i WLASL/start_kit/raw_videos_mp4/XjWSfh50kAU.mp4 -vf scale=1280:720 WLASL/start_kit/raw_videos_mp4/XjWSfh50kAU_fixed.mp4
#ffmpeg -i WLASL/start_kit/raw_videos_mp4/Kwvw-K6GYW8.mp4 -vf scale=1280:720 WLASL/start_kit/raw_videos_mp4/Kwvw-K6GYW8.mp4


1280 720


In [ ]:
#TODO
#BBox pretty good - create new video just using bbox border
#Decide standardized frame and resolution - handle downsampling/upsampling
#Create Processed dataset with 
#Create function to process videos by applying mediapipe to each frame and storing the resulting file in processed folder

"""
Process function
    1. Set location of 30 frame video to test, train, val (Create folder for word if doesn't exist)
    2. check url/id
        - if YT in url, search for YT video download by YT id
        - if id, search for number id
    3. Video exists?
        - if not move on
        - if so, name video after id
    4. Reduce/Increase video to 30 frames
    5. Apply Bbox crop to each video
    6. Save Video
"""